# Problem Set 2 (Total: 64 points)

<b>A Small Intro </b>

For this assignment, we will be using Cook County's (in Illinois) sentencing dataset. This dataset reports the sentence given to defendants convicted of different crimes.

# 0. Load packages and imports

In [1]:
## basic functionality
import pandas as pd
import numpy as np
import re

## plotting
## note: you likely need to install this using
## pip or conda; you can delete this line
## if you're using matplotlib, seaborn, or other
## plotting pkg
import plotnine
from plotnine import *

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## datetime util
from dateutil.relativedelta import relativedelta


## 0.1 Load the data (0 points)

Use `pd.read_csv` to load the `sentencing_asof0405.csv` data (make sure to unzip the `pset2_inputdata` folder and not hard code your user-specific path name)

*Notes*: You may receive a warning about mixed data types upon import; feel free to ignore

## 0.2 Print head, dimensions, info for the data (0 points)

# Part one: data cleaning/interpretation (28 points)

## 1.1: Understanding the unit of analysis (5 points)


### 1.1.1 Print the number of unique values for the following columns. Do so in a way that avoids copying/pasting code for the three (eg apply):

    - Cases (CASE_ID)
    - People (CASE_PARTICIPANT_ID)
    - Charges (CHARGE_ID)


**Sources for this question**: slide 21 here on column-wise apply: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/slides/fall_22/02_ppol564_f22_datawrangling.pdf

### 1.1.2   Write a couple sentences on the following and show an example of each (e.g., a case involving multiple people):
   
- Why there are more unique people than unique cases?
- Why there are more unique charges than unique people?

**Resources**: groupby and agg covered in:
- This activity: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/01_pandas_datacleaning_solutions.ipynb

- These lecture slides: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/slides/fall_22/02_ppol564_f22_datawrangling.pdf


### 1.1.3 Finding mean and median 

- Print the mean and median number of charges per `CASE_PARTICIPANT_ID`

- Print the mean and median number of participants per `CASE_ID`


### 1.1.4 Does the data seem to enable us to follow the same defendant across different cases they're charged in? Write 1 sentence in support of your conclusion.

## 1.2.1 Which offense is final? (3 points)

- First, read the data documentation (link here: https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf) and summarize in your own words the differences between `OFFENSE_CATEGORY` and `UPDATED_OFFENSE_CATEGORY` 

- Construct an indicator `is_changed_offense` that's True for case-participant-charge observations (rows) where there's a difference between the `OFFENSE_CATEGORY` and the `UPDATED_OFFENSE_CATEGORY`. What are some of the more common changed offenses? (can just print result of sort_values based on `OFFENSE_CATEGORY`)

- Print one example of a changed offense from one of these categories and comment on what the reason may be

**Resources**: row subsetting, groupby/agg, and np.where covered in lecture slides
 

## 1.2.2 Simplifying the charges (5 points)

Using the field (`UPDATED_OFFENSE_CATEGORY`), create a new field, `simplified_offense_derived`, that simplifies the many offense categories into broader buckets using the following process:

First, create a new variable that strips "Aggravated" from the UPDATED_OFFENSE_CATEGORY (e.g., Aggravated Battery and Battery just becomes Battery)

**Resources**: slide 25 of this lecture has str.replace (example with stripping the name johnson from a last name): https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/slides/fall_22/02_ppol564_f22_datawrangling.pdf

In [2]:
### first, strip aggravated



Then:
- Combine all offenses with arson in the string into a single arson category (`Arson`)
- Combine all offenses with homicide in the string into a single homicide category (`Homicide`)
- Combine all offenses with vehicle/vehicular in the string into a single vehicle category (`Vehicle-related`)
- Combine all offenses with battery in the string into a single battery category (`Battery`)

Do so efficiently (e.g., using map and a dictionary or np.select rather than separate line for each recoded offense)

**Resources**:
- Activity code and lecture 2 slides cover np.select and map.recode (can use one or the other)

Print the difference between the # of unique offenses in the original `UPDATED_OFFENSE_CATEGORY` field and the # of unique offenses in your new `simplified_offense_derived` field. How many and which ones change?

## 1.3 Cleaning additional variables (10 points)

Clean the following variables; make sure to retain the original variable in data; we tell you to use the derived suffix so it's easier to pull these cleaned out variables later 

### 1.3.1 
- Race: create True/false indicators for `is_black_derived` (Black only or `White/Black [Hispanic or Latino]`), `is_hisp_derived` (Non-Black Hispanic, so either hispanic alone or white hispanic), `is_white_derived` (White non-hispanic), or `is_other_derived` (none of the above)

### 1.3.2
- Gender: create a boolean true/false indicator for `is_male_derived` (false is female, unknown, or other)

### 1.3.3
- Age at incident: you notice outliers like 130-year olds. Recode the top 0.01% of values to be equal to the 99.99th percentile value (this is sometimes called `winsorizing` but don't worry about the terminology). Call this `age_derived`

**Resources**: `np.where` slides linked to above 


### 1.3.4
- Create `sentenceymd_derived` that's a version of `SENTENCING_DATE` converted to datetime format. Also create a rounded version, `sentenceym_derived`, that's rounded down to the first of the month and the year (e.g., 01-05-2016 and 01-27-2016 each become 01-01-2016)
    - Hint: all timestamps are midnight so you can strip the timestamp. Before converting, you'll notice that some of the years have been mistranscribed (e.g., 291X or 221X instead of 201X). Programatically fix those (eg 2914 -> 2014). You can use this regex code to clean the dates or write your own pattern: https://github.com/rebeccajohnson88/PPOL564_slides_activities/issues/20
    
Even after cleaning, there will still be some that are after the year 2021 that we'll filter out later.

**Resources**:

- pd.to_datetime() used in this activity: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/01_pandas_datacleaning_solutions.ipynb 
- extract the month and year from a datetime object using the dt accessor (similar syntax for year): https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.month.html 


### 1.3.5

- Sentencing judge: create an identifier (`judgeid_derived`) for each unique judge (`SENTENCE_JUDGE`) structured as judge_1, judge_2.... When finding unique judges, there are various duplicates we could weed out --- for now, just focus on (1) the different iterations of Doug/Douglas Simpson, (2) the different iterations of Shelley Sutker (who appears both with her maiden name and her hyphenated married name). Note that you can do this more manually by creating a list with the different name variations and receive full credit (eg no need to use regular expressions).
     - Hint: due to mixed types, you may need to cast the `SENTENCE_JUDGE` var to a diff type in order to be able to sort



After finishing, print a random sample of 10 rows with the original and cleaned columns for the relevant variables

**Resources**: sample command here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html

## 1.4 Subsetting rows to analytic dataset (5 points)

You decide based on the above to simplify things in the following ways:
    
- Subset to cases where only one participant is charged, since cases with >1 participant might have complications like 
plea bargains/informing from other participants affecting the sentencing of the focal participant

- To go from a participant-case level dataset, where each participant is repeated across charges tied to the case, to a participant-level dataset, where each participant has one charge, subset to a participant's primary charge and their current sentence (`PRIMARY_CHARGE_FLAG` is True and `CURRENT_SENTENCE_FLAG` is True). Double check that this worked by confirming there are no longer multiple charges for the same case-participant

- Filter out observations where judge is nan or nonsensical (indicated by is.null or equal to FLOOD)

- Subsetting to sentencing date between 01-01-2012 and 04-05-2021 (inclusive)

# Part two: investigating Black vs. White sentencing disparities (36 points)

Now that the data are cleaned, we're going to investigate different types of disparities in sentencing between Black defendants and White defendants. We're focusing on these groups for the purpose of the problem set, but the analysis could be extended to study Hispanic defendants or, in a different jurisdiction, Asian and other minoritized groups.

**Details if interested in digging deeper**: If interested (optional), you can read more technical coverage of how we might (1) measure disparities, and (2) what factors you want to adjust for when deciding whether two defendants are 'similarly situated' but for their race in the following sources:

- [Review of sentencing disparities research](https://www.journals.uchicago.edu/doi/full/10.1086/701505)
- [Discussion of causal model/blinding race at charging stage of the prosecutorial process](https://5harad.com/papers/blind-charging.pdf)
- [Discussion of measuring discrimination in policing that can generalize to the sentencing case](https://www.annualreviews.org/doi/abs/10.1146/annurev-criminol-011518-024731)
- [General discussion of causal challenges in measuring between-group disparities](https://osf.io/preprints/socarxiv/gx4y3/)

**One major caveat**: when comparing whether two similar defendants received different sentences, we're missing one important attribute that influences sentencing: the defendant's criminal history. This influences sentencing both through sentencing guidelines, which can prescribe longer sentences for those who have certain types of prior convictions, and through judicial discretion if judges are more lenient with first-time defendants. The above sources discuss how much we want to "control away" for this prior history, since if we think there are racial biases in which defendants, conditional on *committing* a crime, are arrested and charged, we may not want to adjust for that factor.


## 2.0 (0 points)

First, read in the following dataset (regardless of progress on part one): `sentencing_cleaned.csv`

*Note*: don't worry if there are slight differences in your output from Part One and this dataset/it's not a good use of time to try to reverse engineer Part One answers from this cleaned data.

## 2.1 Investigating one type of between-group difference: who reaches the sentencing stage? (5 points)

Calculate the fraction of Black versus White defendants by month and year

- Denominator is number of unique cases that month
- Numerator for black defendants is count of is_black_derived
- Numerator for white defendants is count of is_white_derived
- Fraction of each is numerator/denominator

**Resources**: groupby and agg code listed above

- With that calculation, create a graph with two lines--- one for Black defendants as fraction of total; another for White defendants. Make sure it includes a legend summarizing which color is for which group, and clean the legend so that it has informative names (e.g., Black or White rather than prop_black or prop_white)
- Use mathematical notation to write out each of the proportions using summation notation in a 1-2 sentence writeup describing trends. What seems to be going on in April and May 2020? 

**Optional challenge (no extra credit points)**: improve the viz by shading the background of the visualization for months with fewer than 100 cases 

**Optional challenge (no extra credit points)**: improve the viz by adding a vertical line for 12-01-2016, the month that new State's Attorney Foxx took office 

## 2.2 Investigating the first type of disparity: probation versus incarceration (10 points)

One type of disparity beyond who arrives at the sentencing stage is whether the defendant receives probation or incaceration.

According to the codebook, incarceration is indicated by `COMMITMENT_TYPE` == "Illinois Department of Corrections"

Recreate the previous plot but where the y axis represents the difference between the following proportions (can be either Black - White or White - Black but make sure to label):

- Percent of black defendants who are incarcerated out of all black defendants that month/year 
- Percent of white defendants who are incarcerated out of all white defendants that month/year 

Briefly interpret the results

## 2.3 Investigating mechanisms: incarceration rates by charge

Your colleague sees the previous graph and is worried that the gap could be different---either wider or smaller---if you adjust for the fact that prosecutors have discretion in what crimes to charge defendants with. If white defendants are charged with crimes that tend to receive probation rather than incarceration, that could explain some of the gaps.

In the next questions, you'll begin to investigate this.

### 2.3.1 Find the most common offenses (3 points)

First, create a set of 'frequent offenses' that represent (over the entire period) the union of the 10 offenses Black defendant are most likely to be charged with and the 10 offenses white defendants are most likely to be charged with (might be far less than 20 total if there's a lot of overlap in common charges)

Use the `simplified_offense_derived` for this

### 2.3.2 Look at incarceration rates (again just whether incarcerated) by race and offense type for these top offenses (3 points)

Print a wide-format version of the resulting table (so each row is an offense type, one col is black incarceration rate for that offense type; another is the white incarceration rate) and interpret. What offenses show the largest disparities in judges being less likely to sentence White defendants to incarceration/more likely to offer those defendants probation?


### 2.3.3 Examine whether this changes pre and post change to charging threshold for retail theft (14 points)

One important question is not only whether there are disparities by offense type but also whether these disparities are changing over time.

The SAO (State Attorney Office), for instance, announced in December of 2016 that they would no longer default to charging retail thefts of under \$1,000 as felonies. This change might have (1) decreased disparities or (2) increased disparities, depending on the correlation between race/ethnicity and magnitude of goods stolen: https://www.dnainfo.com/chicago/20161215/little-village/kim-foxx-raises-bar-for-retail-theft-felonies/. 

Focusing on `simplified_offense_derived` == "Retail theft." Write a user-defined function that allows you to efficiently: 

- Compare Black-White disparities before and after the change using a two-month bandwidth (so pre is October and November 2016; post is January and February 2017)

- Compare Black-White disparities before and after the change using a four-month bandwidth (so pre is August- November 2016; post is January - April 2017)

- Compare Black-White disparities using an eight-month bandwidth

- Compare Black-White disparities using a twelve-month bandwidth

**Note**: Dec. 2016 is always excluded as a transition month

------------------ 

- Print a table with the results (any organization is fine as long as it's clear)  
 
**Resources**:
    
- User-defined function example in lecture on 09/07

- Use that table to create a bar chart where the x axis represents different bandwidths (2, 4, etc); the y axis the size of the Black-White gap, and for each of the x axis points, you have one shaded bar representing "before" the change, another representing "after" the change (make sure that before is ordered before after and the bandwidths are from smallest to largest)

*Note*: for each of the bandwidths include dates spanning the entire month (e.g., for the first, include not only 02-01-2017 but everything up through 02-28-2017; easiest way is for the subsetting to use the rounded `sentenceym_derived`. Also make sure to only include white or black defendants.


**Hint**: depending on how you calculate/reshape things, you may find this issue useful for how to collapse column names with a multilevel index (also may not need it depending on how you structure the code): https://stackoverflow.com/questions/24290297/pandas-dataframe-with-multiindex-column-merge-levels


**Extra credit**: because the bandwidths have different sample sizes, a better viz incorporates measures of uncertainty. Add standard errors to the points using the formula: $(\dfrac{p(1-p)}{n})^{0.5}$ where N is the number of cases in each bandwidth period

### 2.3.4 Interpret the results (1 point)

Write a two-sentence interpretation of the results. What might this show about how people on both sides of the issue---those arguing the policy change will narrow disparities; those arguing the change may widen disparities--could support their claims? 

# 3. Extra credit (2 points)

The above question asked about black-white disparities before and after the policy change. Write a new user-defined function that:

- Has an argument(s) to indicate the levels of the `RACE` variable in the original data that will constitute two groups to compare: group 1 and group 2 (eg in one execution of the function, group 1 might be non-Hispanic white; group 2 might be Hispanic and Black; in another execution, group 1 might be Asian; group 2 but might be Hispanic. Note all levels need to be included in a group)
- Can be used to calculate the same bandwidth-specific disparities as above for defendants in those two groups
- Returns a table or plot with the results
